In [1]:
import requests
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import time
import matplotlib.pyplot as plt
import os
import pickle
import patience_score as ps

key = 'RGAPI-1afb5b52-db74-497f-9165-3ac990d61cec' # api 입력 


In [2]:
grandmaster = 'https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + key
r = requests.get(grandmaster)#그마데이터 호출
league_df = pd.DataFrame(r.json())

league_df.reset_index(inplace=True)#수집한 그마데이터 index정리
league_entries_df = pd.DataFrame(dict(league_df['entries'])).T #dict구조로 되어 있는 entries컬럼 풀어주기
league_df = pd.concat([league_df, league_entries_df], axis=1) #열끼리 결합

league_df = league_df.drop(['index', 'queue', 'name', 'leagueId', 'entries', 'rank'], axis=1)
#league_df.info()

In [3]:
league_df

,tier,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,GRANDMASTER,F3Mh3dSecy9DScll2wOUDwHIjeAKKoNLaVNJINBnt_eRh1Q,상체게임 기도,309,82,64,False,False,True,False
1,GRANDMASTER,KbvjbGkRrM429MkaklO2e96mOVAbJUHZNFgWocK41XWblJQ,우또기,297,127,103,False,False,True,False
2,GRANDMASTER,MNqNKakzyEdF7_Tsw8bOSQ6XOP0P0Sz3zGbxi_iMCPSon2...,eqhgniopqajhpiq,438,126,110,False,False,True,False
3,GRANDMASTER,mSYlT3YaSCTBYVuHbrLh-eFbXYz1H4PfOCp53_mtKFAJ-f...,sossy,369,80,62,False,False,False,False
4,GRANDMASTER,ZZawcPmzZT8yOUqTckW19_musaH6pyjapnl78ywzjnbKB9c,팀개못하면누누함,216,48,36,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...
318,GRANDMASTER,qZr0p_P57x3AEGh4xIse6YVdhyd4EsntSkltt8iURBV4BxZR,VesPene,357,186,162,False,False,True,True
319,GRANDMASTER,DF5oOa3v2Qui3p1pUBNM0gIVvawIJEvhXfZRQk4f2VEEzD...,Forg1ve me,412,76,52,False,False,True,False
320,GRANDMASTER,J1X9-cp5ohuArFUykdWWObqptOZ5MvKP9TQZiIEpKp9MFG8,자기합리화마인드,139,143,131,False,False,False,False
321,GRANDMASTER,oKwMEghbsEKHMOd7GDp3gJlETPTUJRwjG92Cpf4O7S55VQ,Hide on bush,381,153,131,False,False,False,False


In [4]:
gm50=league_df.iloc[0:50] # 그랜드 마스터 데이터 50개 추출.
gm50

,tier,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,GRANDMASTER,F3Mh3dSecy9DScll2wOUDwHIjeAKKoNLaVNJINBnt_eRh1Q,상체게임 기도,309,82,64,False,False,True,False
1,GRANDMASTER,KbvjbGkRrM429MkaklO2e96mOVAbJUHZNFgWocK41XWblJQ,우또기,297,127,103,False,False,True,False
2,GRANDMASTER,MNqNKakzyEdF7_Tsw8bOSQ6XOP0P0Sz3zGbxi_iMCPSon2...,eqhgniopqajhpiq,438,126,110,False,False,True,False
3,GRANDMASTER,mSYlT3YaSCTBYVuHbrLh-eFbXYz1H4PfOCp53_mtKFAJ-f...,sossy,369,80,62,False,False,False,False
4,GRANDMASTER,ZZawcPmzZT8yOUqTckW19_musaH6pyjapnl78ywzjnbKB9c,팀개못하면누누함,216,48,36,False,False,True,False
5,GRANDMASTER,yQ2z0uhNIC6hdUGhrDanSJJM29LSsvz6PdDF-JWgXbsrCwo,nononononononno,420,64,57,False,False,True,False
6,GRANDMASTER,NrhegaHrGoJCk3Wgo0b7EdqTYT08KpC_2aA0CbAWrRk_n6g,AF Krush,242,113,107,False,False,True,False
7,GRANDMASTER,SRaEDzSj15TOBuJSqs5CPXV5NIOdbJZLKHJOuS1n0Up2CQ,레이닝 주인님,263,116,101,False,False,False,False
8,GRANDMASTER,-ELLep6EpZ-TyZnHZU7WxDAIWI_sWIa3PZOztjueSkfTydo,Sup Teacher,265,75,56,False,False,True,True
9,GRANDMASTER,ibTZ40XZXPBeLJHGsQ7Hjmv7GxgcYPrsEXoUc7mtwxLzCNDu,kkverzz,233,146,114,False,False,True,True


In [5]:
#GMummonerId = gm50['summonerId']

GMname =gm50['summonerName']
#GMummonerId
GMname = list(GMname)

GMname

['상체게임 기도',
 '우또기',
 'eqhgniopqajhpiq',
 'sossy',
 '팀개못하면누누함',
 'nononononononno',
 'AF Krush',
 '레이닝 주인님',
 'Sup Teacher',
 'kkverzz',
 '스커지 발사',
 '구거오빠해바라기',
 '원딜민혁 구독',
 '킹 갓 황 샘',
 'shun qi zi ran',
 '10510',
 'nanimonai',
 '변정현개뚱띠',
 'HLE yoHan',
 ' 핑구뿌꾸',
 '킨드돌풍압수',
 '2 2 xh',
 '123qwewqe',
 'tinychild',
 'aldosl',
 'T1 Mirzz',
 '국산 오스카',
 'Liiv Summit',
 'BRO Delight',
 'FA Bud',
 '감당할 수 없이',
 'yIwancibeishang',
 '메가 우기',
 'Dione',
 '빡 표',
 'happy   game ',
 'Liiv Prince1',
 '나무의호흡 제1형',
 'Iknu',
 '저돌맹진저돌맹진',
 'Gen G Karis',
 '무어슬해야할까',
 'AfreecaTV 토마토',
 'BRO Hena',
 'HLE Deft1',
 '짧서라',
 '무우오렌지형1',
 '외 긔레',
 '뀨와룡존버',
 'KAYNKlNG']

In [13]:
GM_result

[94.1176,
 75.0,
 77.2727,
 36.3636,
 74.1935,
 90.0,
 83.3333,
 62.069,
 55.1724,
 48.1481,
 75.6098,
 77.7778,
 76.3158,
 60.7143,
 73.5294,
 61.1111,
 64.8649,
 86.8421,
 58.3333,
 50.0,
 60.0,
 72.973]

In [14]:
i = 5
for name in GMname[22:] :
    try:        
        a = ps.Patience(key, name)
        result = a.get_patience_score()
        GM_result.append(result)
        print(i)
        i = i+1
    except :
        continue

5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [18]:
len(GM_result)

49

In [19]:
with open('gm_result_list.txt', 'wb') as f:
    pickle.dump(GM_result,f)